# 📘 Polars入門：Irisデータセットで学ぶ実践例題集

このノートでは、Pythonの高速データフレームライブラリ **Polars** を使って、Irisデータセットを題材にした**15の実践例題**を通じて、データサイエンスに必要な操作を一通り学びます。また、各例題には **Pandas/dplyrとの対応** も併記しています。


### 🔹 例題1：CSVファイルの読み込み

主な引数:
- `has_header=True`：1行目をヘッダーとして扱う（デフォルト）
- `separator=','`：区切り文字の指定
- `columns=["col1", "col2"]`：読み込む列の指定

**対応**
Pandas: `pd.read_csv(...)`
dplyr: `readr::read_csv(...)`

In [ ]:
import polars as pl
df = pl.read_csv("https://j.mp/iriscsv")

### 🔹 例題2：列の選択

**他の書き方**:
- `df.select(pl.col("sepal_length"))`
- `df.select(pl.col("^sepal_.*$"))`（正規表現で複数列を指定）

**対応**
Pandas: `df[["sepal_length", "species"]]`
dplyr: `df %>% select(sepal_length, species)`

In [ ]:
df.select(["sepal_length", "species"])

### 🔹 例題3：行の選択（フィルタリング）

**対応**
Pandas: `df[df["sepal_length"] > 5.0]`
dplyr: `df %>% filter(sepal_length > 5)`

In [ ]:
df.filter(pl.col("sepal_length") > 5.0)

### 🔹 例題4：列の追加

**対応**
Pandas: `df["double_length"] = df["sepal_length"] * 2`
dplyr: `df %>% mutate(double_length = sepal_length * 2)`

In [ ]:
df.with_columns((pl.col("sepal_length") * 2).alias("double_length"))

### 🔹 例題5：基本統計量の計算

**対応**
Pandas: `df["sepal_length"].mean()`
dplyr: `mean(df$sepal_length)`

In [ ]:
df.select(pl.col("sepal_length").mean())

### 🔹 例題6：グループごとの平均

**対応**
Pandas: `df.groupby("species")["sepal_length"].mean()`
dplyr: `df %>% group_by(species) %>% summarise(mean_sepal_length = mean(sepal_length))`

In [ ]:
df.group_by("species").agg(pl.col("sepal_length").mean().alias("mean_sepal_length"))

### 🔹 例題7：複数列の平均



In [ ]:
df.group_by("species").agg([pl.mean("sepal_length"), pl.mean("sepal_width")])

### 🔹 例題8：全列の平均（`pl.all()`）

`pl.all()` はすべての列を対象にする便利な関数。

In [ ]:
df.group_by("species").agg(pl.all().mean())

### 🔹 例題9：特定列の最大・最小



In [ ]:
df.group_by("species").agg([
    pl.col("sepal_length").max().alias("max_length"),
    pl.col("sepal_length").min().alias("min_length")
])

### 🔹 例題10：CSVへの保存

**対応**
Pandas: `df.to_csv("output.csv")`
dplyr: `readr::write_csv(df, "output.csv")`

In [ ]:
df.write_csv("output.csv")

### 🔹 例題11：JSONへの保存

**対応**
Pandas: `df.to_json(...)`
dplyr: `jsonlite::write_json(...)`

In [ ]:
df.write_json("output.json")

### 🔹 例題12：ファイル読み込み時の列指定

**対応**
Pandas: `pd.read_csv(..., usecols=["sepal_length", "species"])`

In [ ]:
pl.read_csv("https://j.mp/iriscsv", columns=["sepal_length", "species"])

### 🔹 例題13：`concat`による結合



In [ ]:
df1 = pl.DataFrame({"A": [1, 2], "B": [3, 4]})
df2 = pl.DataFrame({"A": [5, 6], "B": [7, 8]})

# 縦方向
pl.concat([df1, df2], how="vertical")

# 横方向
pl.concat([df1, df2], how="horizontal")

### 🔹 例題14：重複の削除

**対応**
Pandas: `df.drop_duplicates()`
dplyr: `distinct(df)` / `distinct(df, species, .keep_all = TRUE)`

In [ ]:
# 完全一致の重複を削除
df.unique()

# 特定の列（species）で重複削除
df.unique(subset=["species"])

### 🔹 例題15：複数の処理をまとめて実行

Polarsは**メソッドチェーン**で複数処理を直感的に書けるのが強み。

In [ ]:
df = pl.read_csv("https://j.mp/iriscsv")

df_agg = (
    df
    .select([pl.col("^petal_.*$"), pl.col("species")])
    .with_columns((pl.col("petal_length") * 1.5).alias("scaled_petal_length"))
    .filter(pl.col("petal_width") > 1.0)
    .group_by("species")
    .agg(pl.all().median())
)

## ✏️ おわりに

Polarsは、高速・安全・柔軟なデータ処理ができるPythonライブラリです。このノートでは、Irisデータセットを通じて、**実務でよく使う処理を例題形式で習得**できるよう構成しました。

今後の応用：
- 欠損値処理
- ウィンドウ関数
- ジョイン・結合
- 時系列処理

などもPolarsで非常に簡潔に記述できます。